In [1]:
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import json 
from geopy.geocoders import Nominatim

import geocoder
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans 
import folium 
import wget as wget

In [2]:
result = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
src=result.content
soup= BeautifulSoup(src,'html.parser')
table = soup.find('table',{'wikitable sortable'})

In [3]:
rows = table.find_all("tr")
l=[]
for r in rows:
    td=r.find_all("td")
    rw=[r.text for r in td]
    l.append(rw)
Canada=pd.DataFrame(l,columns=["Postcode","Borough","Neighbourhood"])

In [4]:
Canada

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
...,...,...,...
283,M8Z,Etobicoke,Mimico NW\n
284,M8Z,Etobicoke,The Queensway West\n
285,M8Z,Etobicoke,Royal York South West\n
286,M8Z,Etobicoke,South of Bloor\n


In [5]:
Canada=Canada[Canada.Borough!='Not assigned']

In [6]:
Canada['Neighbourhood']=Canada['Neighbourhood'].replace(to_replace='Not assigned\n',value=Canada['Borough'])

In [7]:
Canada = Canada.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join }).reset_index()

In [8]:
def get_details(postal):
    lat_long= None
    while(lat_long is None):
         geo=geocoder.arcgis('{},Toronto,Ontario'.format(postal))
         lat_long = geo.latlng
    return lat_long

In [9]:
cod=Canada['Postcode']
Coordinate=[get_details(cod) for cod in cod.tolist()]

Cords=pd.DataFrame(Coordinate, columns=['Latitude','Longitude'])
Canada['Latitude']=Cords['Latitude']
Canada['Longitude']=Cords['Longitude']

In [10]:
Canada.drop(['Postcode'],axis= 1,inplace=True)
Canada 

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Rouge\n, Malvern\n",43.811525,-79.195517
1,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n",43.785665,-79.158725
2,Scarborough,"Guildwood\n, Morningside\n, West Hill\n",43.765815,-79.175193
3,Scarborough,Woburn\n,43.768369,-79.217590
4,Scarborough,Cedarbrae\n,43.769688,-79.239440
...,...,...,...,...
98,York,Weston\n,43.704845,-79.517546
99,Etobicoke,Westmount\n,43.696505,-79.530252
100,Etobicoke,"Kingsview Village\n, Martin Grove Gardens\n, R...",43.686810,-79.557284
101,Etobicoke,"Albion Gardens\n, Beaumond Heights\n, Humberga...",43.743145,-79.584664


In [11]:
with open('C:/Users/acer/.anaconda/navigator/nyu_2451_34572-geojson.json') as json_data:
                 newyork_data = json.load(json_data)

In [12]:
neighborhoods_data = newyork_data['features']

In [13]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

new_york = pd.DataFrame(columns=column_names)

In [14]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    new_york = new_york.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [22]:
new_york=new_york.iloc[:153,:]
new_york

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
...,...,...,...,...
148,Queens,South Ozone Park,40.668550,-73.809865
149,Queens,College Point,40.784903,-73.843045
150,Queens,Whitestone,40.781291,-73.814202
151,Queens,Bayside,40.766041,-73.774274


In [23]:
CLIENT_ID = 'H0ODNDLWAN12HG3ZXXGWMBLU0AV1SXIS0E0ILO4DL5ZNVPK1'
CLIENT_SECRET = 'GZGTBFWNX1Y1XMA4Z2P1KWJQLSFI24L1FNLU4FI0UJCCMRS0' 
VERSION = '20191204' 

In [24]:
def getNearbyVenues(code,names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for codes,name, lat, lng in zip(code,names, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/explore?ll={},{}&client_id={}&client_secret={}&v={}&radius={}&limit={}'.format(
            lat, 
            lng,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            500, 
            40)
            
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            codes,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough','Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
canadian_venues = getNearbyVenues(code=Canada['Borough'],
                                    names=Canada['Neighbourhood'],
                                   latitudes=Canada['Latitude'],
                                   longitudes=Canada['Longitude']
                                  )
canadian_venues

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,"Rouge\n, Malvern\n",43.811525,-79.195517,Straits Aqualife Limited,43.809268,-79.199399,Pet Store
1,Scarborough,"Rouge\n, Malvern\n",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
2,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n",43.785665,-79.158725,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,"Guildwood\n, Morningside\n, West Hill\n",43.765815,-79.175193,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
4,Scarborough,"Guildwood\n, Morningside\n, West Hill\n",43.765815,-79.175193,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
...,...,...,...,...,...,...,...,...
1599,Etobicoke,Northwest\n,43.711740,-79.579181,Subway,43.713345,-79.580358,Sandwich Place
1600,Etobicoke,Northwest\n,43.711740,-79.579181,Petro-Canada,43.714398,-79.581868,Gas Station
1601,Etobicoke,Northwest\n,43.711740,-79.579181,U-Haul Moving & Storage at Rexdale Blvd,43.713500,-79.577307,Storage Facility
1602,Etobicoke,Northwest\n,43.711740,-79.579181,Fitness Two 4,43.714482,-79.582903,Gym


In [25]:
newyork_venues = getNearbyVenues( code=new_york['Borough'],
                                   names=new_york['Neighborhood'],
                                   latitudes=new_york['Latitude'],
                                   longitudes=new_york['Longitude']
                                  )
newyork_venues 

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bronx,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Bronx,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Bronx,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Bronx,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Bronx,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop
...,...,...,...,...,...,...,...,...
4321,Queens,Auburndale,40.761730,-73.791762,Food Universe Marketplace,40.764321,-73.787178,Supermarket
4322,Queens,Auburndale,40.761730,-73.791762,Park City Gymnastics,40.761572,-73.795671,Gymnastics Gym
4323,Queens,Auburndale,40.761730,-73.791762,Robert's Prime Meats and Deli,40.762814,-73.795820,Deli / Bodega
4324,Queens,Auburndale,40.761730,-73.791762,Breifne Pub,40.762738,-73.796928,Bar


In [26]:
canadian_venues.groupby(["Borough"]).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,,
Central Toronto,74,74,74,74,74,74,74
Downtown Toronto,619,619,619,619,619,619,619
East Toronto,108,108,108,108,108,108,108
East York,82,82,82,82,82,82,82
Etobicoke,73,73,73,73,73,73,73
Mississauga,40,40,40,40,40,40,40
North York,261,261,261,261,261,261,261
Queen's Park,48,48,48,48,48,48,48
Scarborough,90,90,90,90,90,90,90


In [28]:
newyork_venues.groupby(["Borough"]).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,,
Bronx,900,900,900,900,900,900,900
Brooklyn,1515,1515,1515,1515,1515,1515,1515
Manhattan,1171,1171,1171,1171,1171,1171,1171
Queens,740,740,740,740,740,740,740


In [29]:
canada_onehot = pd.get_dummies(canadian_venues[['Venue Category']], prefix="", prefix_sep="")

canada_onehot['Borough'] = canadian_venues['Borough'] 

fixed_columns = [canada_onehot.columns[-1]] + list(canada_onehot.columns[:-1])
canada_onehot = canada_onehot[fixed_columns]

canada_onehot.head()

,Borough,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
canada_grouped = canada_onehot.groupby('Borough').mean().reset_index()
canada_grouped

,Borough,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.013514,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.013514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Downtown Toronto,0.000000,0.019386,0.001616,0.012924,0.001616,0.008078,0.003231,0.000000,0.000000,...,0.001616,0.001616,0.008078,0.001616,0.000000,0.006462,0.008078,0.001616,0.000000,0.001616
2,East Toronto,0.000000,0.027778,0.009259,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.009259,0.000000,0.000000,0.009259,0.000000,0.000000,0.000000,0.000000
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.012195,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.013699,0.013699,0.013699,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.000000,0.000000,0.000000,0.000000
5,Mississauga,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.003831,0.007663,0.000000,0.000000,0.003831,0.003831,0.000000,0.000000,0.003831,...,0.000000,0.000000,0.000000,0.003831,0.000000,0.011494,0.000000,0.003831,0.000000,0.000000
7,Queen's Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,...,0.011111,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000
9,West Toronto,0.000000,0.016760,0.000000,0.005587,0.005587,0.016760,0.005587,0.000000,0.000000,...,0.000000,0.000000,0.016760,0.000000,0.000000,0.011173,0.005587,0.000000,0.000000,0.005587


In [31]:
ny_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

ny_onehot['Borough'] = newyork_venues['Borough'] 

fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Borough,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Waste Facility,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bronx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
ny_grouped = ny_onehot.groupby('Borough').mean().reset_index()
ny_grouped

,Borough,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Vietnamese Restaurant,Warehouse Store,Waste Facility,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Bronx,0.000000,0.000000,0.002222,0.011111,0.001111,0.001111,0.001111,0.000000,0.002222,...,0.000000,0.001111,0.001111,0.000000,0.000000,0.000000,0.001111,0.002222,0.002222,0.000000
1,Brooklyn,0.000660,0.000000,0.000000,0.013861,0.001980,0.000000,0.001320,0.000660,0.005281,...,0.001320,0.000000,0.000000,0.000660,0.000000,0.006601,0.015842,0.000000,0.001980,0.014521
2,Manhattan,0.002562,0.000854,0.002562,0.026473,0.000000,0.000854,0.000000,0.001708,0.007686,...,0.004270,0.000000,0.000000,0.000854,0.000854,0.011102,0.013664,0.000854,0.006832,0.010248
3,Queens,0.000000,0.000000,0.000000,0.014865,0.000000,0.000000,0.001351,0.002703,0.000000,...,0.004054,0.000000,0.000000,0.000000,0.000000,0.001351,0.001351,0.000000,0.000000,0.005405


In [33]:
num_top_venues = 3

for hood in canada_grouped['Borough']:
    print("----"+hood+"----")
    temp = canada_grouped[canada_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.07
2            Park  0.05


----Downtown Toronto----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.07
2   Restaurant  0.04


----East Toronto----
              venue  freq
0        Steakhouse  0.05
1       Pizza Place  0.05
2  Sushi Restaurant  0.05


----East York----
                 venue  freq
0          Coffee Shop  0.06
1  Sporting Goods Shop  0.04
2       Breakfast Spot  0.04


----Etobicoke----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.05
2   Grocery Store  0.05


----Mississauga----
        venue  freq
0  Steakhouse  0.10
1        Café  0.08
2       Hotel  0.05


----North York----
                  venue  freq
0        Clothing Store  0.07
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.06


----Queen's Park----
            venue  freq
0     Coffee Shop  0.29
1  Sandwich Place  0.08
2        Pharmacy  0.06


----Scarborough----
      

In [34]:
num_top_venues = 3

for hood in ny_grouped['Borough']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bronx----
           venue  freq
0    Pizza Place  0.09
1     Donut Shop  0.04
2  Deli / Bodega  0.04


----Brooklyn----
         venue  freq
0  Pizza Place  0.05
1  Coffee Shop  0.03
2          Bar  0.03


----Manhattan----
         venue  freq
0  Coffee Shop  0.04
1         Café  0.03
2         Park  0.03


----Queens----
         venue  freq
0  Pizza Place  0.04
1       Bakery  0.04
2     Pharmacy  0.03




In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

can_sorted = pd.DataFrame(columns=columns)
can_sorted['Borough'] = canada_grouped['Borough']

for ind in np.arange(canada_grouped.shape[0]):
    can_sorted.iloc[ind, 1:] = return_most_common_venues(canada_grouped.iloc[ind, :], num_top_venues)

can_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Central Toronto,Sandwich Place,Coffee Shop,Café,Gym,Park
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Bakery,Hotel
2,East Toronto,Coffee Shop,Pizza Place,Steakhouse,Café,Sushi Restaurant
3,East York,Coffee Shop,Bus Line,Grocery Store,Sporting Goods Shop,Breakfast Spot
4,Etobicoke,Coffee Shop,Grocery Store,Sandwich Place,Gym,Burrito Place
5,Mississauga,Steakhouse,Café,Hotel,Concert Hall,Pizza Place
6,North York,Coffee Shop,Clothing Store,Fast Food Restaurant,Pizza Place,Park
7,Queen's Park,Coffee Shop,Sandwich Place,Café,Pharmacy,Park
8,Scarborough,Chinese Restaurant,Coffee Shop,Pharmacy,Discount Store,Grocery Store
9,West Toronto,Coffee Shop,Bakery,Café,Bar,Park


In [38]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

ny_sorted = pd.DataFrame(columns=columns)
ny_sorted['Borough'] = ny_grouped['Borough']

for ind in np.arange(ny_grouped.shape[0]):
    ny_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

ny_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bronx,Pizza Place,Deli / Bodega,Donut Shop,Italian Restaurant,Sandwich Place
1,Brooklyn,Pizza Place,Coffee Shop,Bar,Italian Restaurant,Deli / Bodega
2,Manhattan,Coffee Shop,Café,Park,Italian Restaurant,American Restaurant
3,Queens,Pizza Place,Bakery,Chinese Restaurant,Pharmacy,Italian Restaurant


In [40]:
kclusters = 3

canada_clustering = canada_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(canada_clustering)

kmeans.labels_[0:10] 

array([0, 0, 0, 1, 1, 0, 1, 2, 1, 0])

In [41]:
can_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

canada_merged = Canada

canada_merged = canada_merged.join(can_sorted.set_index('Borough'), on='Borough')

In [42]:
canada_merged=canada_merged.dropna()
canada_merged.reset_index(inplace=True)

In [43]:
canada_merged

,index,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,Scarborough,"Rouge\n, Malvern\n",43.811525,-79.195517,1,Chinese Restaurant,Coffee Shop,Pharmacy,Discount Store,Grocery Store
1,1,Scarborough,"Highland Creek\n, Rouge Hill\n, Port Union\n",43.785665,-79.158725,1,Chinese Restaurant,Coffee Shop,Pharmacy,Discount Store,Grocery Store
2,2,Scarborough,"Guildwood\n, Morningside\n, West Hill\n",43.765815,-79.175193,1,Chinese Restaurant,Coffee Shop,Pharmacy,Discount Store,Grocery Store
3,3,Scarborough,Woburn\n,43.768369,-79.217590,1,Chinese Restaurant,Coffee Shop,Pharmacy,Discount Store,Grocery Store
4,4,Scarborough,Cedarbrae\n,43.769688,-79.239440,1,Chinese Restaurant,Coffee Shop,Pharmacy,Discount Store,Grocery Store
...,...,...,...,...,...,...,...,...,...,...,...
98,98,York,Weston\n,43.704845,-79.517546,1,Fast Food Restaurant,Brewery,Park,Gym,Coffee Shop
99,99,Etobicoke,Westmount\n,43.696505,-79.530252,1,Coffee Shop,Grocery Store,Sandwich Place,Gym,Burrito Place
100,100,Etobicoke,"Kingsview Village\n, Martin Grove Gardens\n, R...",43.686810,-79.557284,1,Coffee Shop,Grocery Store,Sandwich Place,Gym,Burrito Place
101,101,Etobicoke,"Albion Gardens\n, Beaumond Heights\n, Humberga...",43.743145,-79.584664,1,Coffee Shop,Grocery Store,Sandwich Place,Gym,Burrito Place


In [44]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

ny_sorted = pd.DataFrame(columns=columns)
ny_sorted['Borough'] = ny_grouped['Borough']

for ind in np.arange(ny_grouped.shape[0]):
    ny_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

ny_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bronx,Pizza Place,Deli / Bodega,Donut Shop,Italian Restaurant,Sandwich Place
1,Brooklyn,Pizza Place,Coffee Shop,Bar,Italian Restaurant,Deli / Bodega
2,Manhattan,Coffee Shop,Café,Park,Italian Restaurant,American Restaurant
3,Queens,Pizza Place,Bakery,Chinese Restaurant,Pharmacy,Italian Restaurant


In [46]:
kclusters = 3

ny_clustering = ny_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_clustering)

kmeans.labels_[0:4] 

array([0, 1, 2, 1])

In [47]:
ny_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = new_york

ny_merged = ny_merged.join(ny_sorted.set_index('Borough'), on='Borough')

In [48]:
ny_merged=ny_merged.dropna()
ny_merged.reset_index(inplace=True)

In [49]:
ny_merged

,index,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,0,Bronx,Wakefield,40.894705,-73.847201,0,Pizza Place,Deli / Bodega,Donut Shop,Italian Restaurant,Sandwich Place
1,1,Bronx,Co-op City,40.874294,-73.829939,0,Pizza Place,Deli / Bodega,Donut Shop,Italian Restaurant,Sandwich Place
2,2,Bronx,Eastchester,40.887556,-73.827806,0,Pizza Place,Deli / Bodega,Donut Shop,Italian Restaurant,Sandwich Place
3,3,Bronx,Fieldston,40.895437,-73.905643,0,Pizza Place,Deli / Bodega,Donut Shop,Italian Restaurant,Sandwich Place
4,4,Bronx,Riverdale,40.890834,-73.912585,0,Pizza Place,Deli / Bodega,Donut Shop,Italian Restaurant,Sandwich Place
...,...,...,...,...,...,...,...,...,...,...,...
148,148,Queens,South Ozone Park,40.668550,-73.809865,1,Pizza Place,Bakery,Chinese Restaurant,Pharmacy,Italian Restaurant
149,149,Queens,College Point,40.784903,-73.843045,1,Pizza Place,Bakery,Chinese Restaurant,Pharmacy,Italian Restaurant
150,150,Queens,Whitestone,40.781291,-73.814202,1,Pizza Place,Bakery,Chinese Restaurant,Pharmacy,Italian Restaurant
151,151,Queens,Bayside,40.766041,-73.774274,1,Pizza Place,Bakery,Chinese Restaurant,Pharmacy,Italian Restaurant


In [53]:
address = 'Toronto'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

canada_merged['Cluster Labels']=canada_merged['Cluster Labels'].astype(int)
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.5)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(canada_merged['Latitude'], canada_merged['Longitude'], canada_merged['Borough'], canada_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
address = 'New York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

ny_merged['Cluster Labels']=ny_merged['Cluster Labels'].astype(int)
ma_clusters = folium.Map(location=[latitude, longitude], zoom_start=10.5)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Borough'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(ma_clusters)
       
ma_clusters

# End of Capstone Project